# Generalized Mie Near Field Plots

Chad Heaps
February 25, 2017

This was originally a separate utility, but is being added to mstm-scan to aid anyone running the calculations.  The saved scattered coefficient file should be the correct format to be read into the notebook.

This jupyter notebook plots the scattered near field from a Generalized Mie calculation.  The vector components are calculated using Fortran code from Daniel Mackowski's MSTM code (http://www.eng.auburn.edu/~dmckwski/scatcodes/).  The Fortran code is compiled using fortranmagic.  You could easily compile it with f2py separately, too.  There is a Fortran module func_nf and a python class NearField that take care of most of the computation.  They are mostly stitched together from other scripts/codes.

Only Fields in the Cartesian planes are currently supported.  

The code requires a text file with vector spherical harmonic coefficients and some header information.  Right now it will only calculate fields external to the particles.

The field for each particle is calculated individually and summed at the end.  The field is  expressed as

\begin{equation}
T = \sum_{l=i}^N \frac{1}{2} m_i v_i^2
\end{equation}

where the explicit forms for ${\bf N}_{lm}$ and ${\bf M}_{lm}$ may be found in the MSTM manual or other Mie theory resources.

In [1]:
import time
import re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
try: 
    %load_ext fortranmagic
except:
    %install_ext https://raw.github.com/mgaitan/fortran_magic/master/fortranmagic.py
    %load_ext fortranmagic